In [2]:
import numpy as np
import copy

In [ ]:
def añadir_variables_artificiales(A, b):
    vars = []
    for v in b:
        if v >= 0:
            vars.append(1)
        else:
            vars.append(-1)
    array = np.array(vars)
    matriz_diagonal = np.diag(array)
    # Usamos np.hstack para apilar horizontalmente
    resultado = np.hstack((A, matriz_diagonal))
    return resultado


In [ ]:
def SimplexFaseII(A, b, c, B_ini, B_indices,B_inv):
    bool_acotado = True
    m,n = A.shape
    N_ini = set(i for i in range(n) if i not in B_indices) 
    
    x_ini = np.zeros(n) 
    x_b = B_inv @ b 
    x_ini[B_indices] = x_b # Asignar los valores de x_b en las posiciones correspondientes

    def actualizacion_inversa(d,inv_anterior,p):
        d = d.T.tolist()
        vector_n = np.array([[-d[i]/d[p] if i != p else -1/d[p] for i in range(len(d))]])
        e = np.identity(len(d))
        e[p] = vector_n
        e = e.T
        inversa = e @ inv_anterior 
        return inversa

    iter = 1
    while True:#####################################################################################
        
        B_may = A[:, B_indices] 
        N_ini = sorted(N_ini)
        A_n = A[:, N_ini]
        cn = np.array([c[i] for i in N_ini])
        cb = np.array([c[i] for i in B_indices])
        z = cb @ x_b

        #costes reducidos
        r = cn - cb @ B_inv @ A_n 
        if np.all(r >= 0):
            return B_indices, z, bool_acotado, B_may, B_inv
        i = 0
        minimo = float('inf')
        for j in range(len(r)):
            if r[j] < minimo:# Deberiamos elegir el del indice menor pero como N_ini esta ordenado ya habriamos elegido el del menor indice
                minimo = r[j]
                i = j
            
        q = N_ini[i] # variable q entra en la base 
        
        #direccion de descenso 
        d_b = -B_inv @ A[:, q]
        if np.all(d_b >= 0):
            print("P.L. NO ACOTADO!")
            bool_acotado = False
            return None, None, bool_acotado, None, None
        d = np.zeros(n)  
        d[B_indices] = d_b
        d[q] = 1

        #longitud de paso
        p = 0
        theta = float('inf')
        for w in range(len(d_b)):
            if d_b[w] < 0:
                val = -x_b[w]/d_b[w]
                if val < theta:
                    theta, p = val, w
                elif val == theta:  # Si hay empate, elegimos el menor índice en B_indices (regla de Bland)
                    if B_indices[w] < B_indices[p]:  
                        p = w

        #print(f"Iteracion {iter}: q = {q}, B[p] = {B_indices[p]}, theta* = {theta}, xb = {x_b} z = {z}")
        #print(f"Iteracion {iter}: q = {q}, B[p] = {B_indices[p]}, theta* = {theta}, xb = {[round(x, 4) for x in x_b]} z = {z}")
        print(f"Iteracion {iter}: q = {q}, B[p] = {B_indices[p]}, theta* = {round(float(theta), 5)}, z = {round(float(z), 5)}")
        print("xb:", [round(float(x), 4) for x in x_b])
        print()
        #Actualizaciones
        x_ini = x_ini + theta*d
        z = z + theta * r[i] 
        var_saliente, B_indices[p] = B_indices[p], q
        N_ini.remove(q) 
        N_ini.append(var_saliente)
        x_b = x_ini[B_indices]
        B_inv = actualizacion_inversa(d_b,B_inv,p)
    
        iter += 1


In [6]:
def SimplexFaseI(A, b, c):
    print("Inicio del Simplex Primal con la regla de Bland")
    
    m, n = A.shape #las filas son m (el numero de restr.) y n son las columnas (num de variables)
    A_inicial = añadir_variables_artificiales(A, b)
    b_inicial = b 
    c_inicial = [0]*n + [1]*m
    B_inicial = A_inicial[:,n:]
    B_inversa = B_inicial
    B_indices = list(i for i in range(n, n+m)) 
    print("Fase I")
    B_indices_II, z_inicial, f,B_ini_II, B_inversa  = SimplexFaseII(A_inicial,b_inicial,c_inicial,B_inicial, B_indices, B_inversa)
    if (z_inicial != 0):
        print("P.L NO FACTIBLE! z no es 0 despues de la fase inicial")
        return None,None,None,False 
    print("Fase II")
    
    # poner if para vigilar que no hay variables artificiales entre los indices de la base B_indices_II

    B_indices_opt, z , bool_acotado, B_optima, _ = SimplexFaseII(A,b,c,B_ini_II,B_indices_II,B_inversa)
    
    return B_indices_opt, z , bool_acotado, True



Soluciones:

In [25]:
c = np.array([62, 18, -34, -88, -97, -24, 19, -15, 31, -13, -63, 81, -3, 63, 0, 0, 0, 0, 0, 0])

# Matriz de restricciones
A = np.array([
    [-42, -35, 68, 50, 73, 64, -44, -4, 36, -45, -89, 88, 75, -44, 0, 0, 0, 0, 0, 0],
    [49, -51, 32, 73, 32, -96, 33, 20, 61, -77, 97, 4, -56, 3, 0, 0, 0, 0, 0, 0],
    [21, -26, -86, -68, 31, 1, -10, 84, 13, -62, -1, 10, 68, 96, 0, 0, 0, 0, 0, 0],
    [49, 92, -39, 66, -89, -5, 95, 55, -24, 71, 7, 14, 23, 70, 0, 0, 0, 0, 0, 0],
    [73, 61, 93, 96, 66, 99, 76, 53, 78, 55, 91, 68, 68, 54, 1, 0, 0, 0, 0, 0],
    [-29, 17, -42, 94, -60, 28, 21, 74, 99, 31, -15, 6, -29, -15, 0, 1, 0, 0, 0, 0],
    [-97, 51, -91, 41, -57, 16, 92, 81, -35, 36, 72, -6, 22, 14, 0, 0, 1, 0, 0, 0],
    [-76, 27, 17, 69, 80, -51, 5, 65, -87, 6, -72, 20, -78, 88, 0, 0, 0, -1, 0, 0],
    [69, -1, 44, 20, -81, 27, 71, -41, 85, 91, -58, 7, -41, -30, 0, 0, 0, 0, 1, 0],
    [-5, 87, -60, 15, 94, 65, 76, 69, 62, -20, -14, 68, 53, 96, 0, 0, 0, 0, 0, -1]
])

# Términos independientes de las restricciones
b = np.array([151, 124, 71, 385, 1032, 181, 140, 12, 163, 585])

opt, zeta, acotado, factible = SimplexFaseI(A,b,c)
if acotado and factible:
    print("vb:",opt)
    print("z*:",zeta)

Inicio del Simplex Primal con la regla de Bland
Fase I
Iteracion 1: q = 3, B[p] = 27, theta* = 0.17391, z = 2844.0
xb: [151.0, 124.0, 71.0, 385.0, 1032.0, 181.0, 140.0, 12.0, 163.0, 585.0]

Iteracion 2: q = 8, B[p] = 21, theta* = 0.72727, z = 2764.69565
xb: [142.3043, 111.3043, 82.8261, 373.5217, 1015.3043, 164.6522, 132.8696, 0.1739, 159.5217, 582.3913]

Iteracion 3: q = 5, B[p] = 25, theta* = 0.04105, z = 2137.09091
xb: [70.2727, 0.7273, 135.7273, 330.4545, 870.5455, 6.4545, 120.7273, 1.0909, 79.3636, 523.5455]

Iteracion 4: q = 6, B[p] = 28, theta* = 1.24964, z = 2107.448
xb: [65.0115, 0.7385, 138.5697, 327.9895, 861.324, 0.0411, 118.6382, 1.1355, 76.4055, 519.5096]

Iteracion 5: q = 10, B[p] = 26, theta* = 0.00343, z = 1680.79166
xb: [121.2678, 0.5673, 142.3164, 216.6239, 775.0971, 0.2413, 0.971, 0.977, 1.2496, 424.5154]

Iteracion 6: q = 4, B[p] = 20, theta* = 1.22076, z = 1678.2077
xb: [121.5429, 0.5637, 142.5421, 215.9438, 774.0767, 0.2456, 0.0034, 0.9789, 1.2545, 424.1022]

Ite

In [26]:
# Vector c
c1 = np.array([-28, -4, 33, 51, -35, -51, -8, -49, 1, 3, -71, -20, -64, -68, 0, 0, 0, 0, 0, 0])

# Matriz A
A1 = np.array([
    [78, 92, 5, 49, -89, 45, -31, 64, 89, 61, 12, 6, -35, -88, 0, 0, 0, 0, 0, 0],
    [65, -100, 97, -31, 65, 64, 41, 39, -36, 64, 91, -84, -100, -40, 0, 0, 0, 0, 0, 0],
    [51, -15, -36, 56, -9, -7, 19, 67, -23, 88, -2, 93, -74, -38, 0, 0, 0, 0, 0, 0],
    [-25, -16, 11, 88, 70, -69, -19, -8, 63, -6, 19, -98, 75, -30, 0, 0, 0, 0, 0, 0],
    [74, 67, 53, 55, 90, 96, 76, 68, 68, 70, 86, 84, 94, 97, 1, 0, 0, 0, 0, 0],
    [-54, 6, -72, 65, 71, 12, 0, -62, -60, 56, 82, 81, -83, -6, 0, 1, 0, 0, 0, 0],
    [59, 48, -16, 61, -65, -58, -3, -46, 80, 74, 74, 74, -8, 1, 0, 0, 1, 0, 0, 0],
    [-42, 97, -53, -74, 55, -5, 10, -99, 58, -44, 47, 66, 0, -4, 0, 0, 0, -1, 0, 0],
    [-25, 20, 57, 45, 56, 90, -6, -73, -76, 95, 67, -27, -30, -65, 0, 0, 0, 0, -1, 0],
    [98, 80, -58, 81, -4, -13, -24, -24, 7, -71, 38, 44, 71, -69, 0, 0, 0, 0, 0, -1]
])

# Vector b
b1 = np.array([258, 135, 170, 55, 1079, 37, 276, 11, 127, 155])
opt, zeta, acotado, factible = SimplexFaseI(A1,b1,c1)
if acotado and factible:
    print("vb:",opt)
    print("z*:",zeta)


Inicio del Simplex Primal con la regla de Bland
Fase I
Iteracion 1: q = 10, B[p] = 27, theta* = 0.23404, z = 2303.0
xb: [258.0, 135.0, 170.0, 55.0, 1079.0, 37.0, 276.0, 11.0, 127.0, 155.0]

Iteracion 2: q = 3, B[p] = 25, theta* = 0.09175, z = 2182.70213
xb: [255.1915, 113.7021, 170.4681, 50.5532, 1058.8723, 17.8085, 258.6809, 0.234, 111.3191, 146.1064]

Iteracion 3: q = 0, B[p] = 21, theta* = 0.76498, z = 2072.2144
xb: [248.9625, 103.4012, 165.6192, 39.735, 1041.4035, 0.0917, 242.3949, 0.3785, 97.5123, 133.1858]

Iteracion 4: q = 1, B[p] = 29, theta* = 0.14869, z = 1598.9055
xb: [186.249, 0.765, 131.9876, 54.829, 940.4708, 0.0158, 160.1605, 0.9425, 82.2684, 42.9402]

Iteracion 5: q = 9, B[p] = 3, theta* = 0.75252, z = 1432.94534
xb: [150.3224, 0.9777, 117.643, 52.4919, 905.0226, 0.1197, 130.6671, 0.9893, 76.7984, 0.1487]

Iteracion 6: q = 2, B[p] = 28, theta* = 0.07522, z = 1081.88681
xb: [41.2192, 1.3014, 48.7486, 88.3774, 821.8434, 0.7525, 56.7862, 0.7463, 24.912, 0.6566]

Iteracion 

In [27]:
c2 = np.array([83, -62, -65, -49, 27, -78, -73, -83, -29, -17, -43, 42, 76, 2, 0, 0, 0, 0, 0, 0])

# Matriz A
A2 = np.array([
    [3, -8, -66, 27, -45, 0, -47, 99, -5, 66, 71, -64, 86, 51, 0, 0, 0, 0, 0, 0],
    [16, 93, -5, 88, -56, 81, 28, 10, 43, -12, 98, 14, -54, 1, 0, 0, 0, 0, 0, 0],
    [-82, 36, 45, 61, -50, -90, -38, 32, 8, -21, 50, 84, -6, 80, 0, 0, 0, 0, 0, 0],
    [-28, 18, 72, -86, -13, 91, 37, 15, 91, 98, 69, -97, -2, 20, 0, 0, 0, 0, 0, 0],
    [19, -50, 39, 61, -23, -16, -35, 68, 18, 51, 56, -13, -69, 81, 1, 0, 0, 0, 0, 0],
    [-84, 42, -73, -57, 91, -50, -32, 47, 86, -59, 35, 21, 48, 53, 0, -1, 0, 0, 0, 0],
    [52, 39, 22, -49, -36, -39, 67, 32, 48, 77, 66, -51, -2, -94, 0, 0, 1, 0, 0, 0],
    [66, 91, 0, 14, 4, 56, 35, 15, 15, 10, 20, 63, 73, 57, 0, 0, 0, 1, 0, 0],
    [27, 21, 98, 38, 23, 33, 91, 38, 59, 69, 21, 53, 87, 41, 0, 0, 0, 0, 1, 0],
    [5, 69, 83, 55, 12, 68, 25, 28, 28, 62, 14, 27, 87, 53, 0, 0, 0, 0, 0, 1]
])

# Vector b
b2 = np.array([168, 345, 109, 285, 188, 67, 133, 30, 80, 62])

opt, zeta, acotado, factible = SimplexFaseI(A2,b2,c2)
if acotado and factible:
    print("vb:",opt)
    print("z*:",zeta)

Inicio del Simplex Primal con la regla de Bland
Fase I
Iteracion 1: q = 10, B[p] = 27, theta* = 1.5, z = 1467.0
xb: [168.0, 345.0, 109.0, 285.0, 188.0, 67.0, 133.0, 30.0, 80.0, 62.0]

Iteracion 2: q = 2, B[p] = 29, theta* = 0.49398, z = 717.0
xb: [61.5, 198.0, 34.0, 181.5, 104.0, 14.5, 34.0, 1.5, 48.5, 41.0]

Iteracion 3: q = 14, B[p] = 24, theta* = 84.73494, z = 610.79518
xb: [94.1024, 200.4699, 11.7711, 145.9337, 84.7349, 50.5602, 23.1325, 1.5, 0.0904, 0.494]

Iteracion 4: q = 8, B[p] = 28, theta* = 0.004, z = 526.06024
xb: [94.1024, 200.4699, 11.7711, 145.9337, 84.7349, 50.5602, 23.1325, 1.5, 0.0904, 0.494]

Iteracion 5: q = 16, B[p] = 26, theta* = 23.15709, z = 526.04867
xb: [94.2798, 200.5877, 11.9271, 145.8374, 84.8638, 50.2596, 23.1571, 1.497, 0.004, 0.4931]

Iteracion 6: q = 18, B[p] = 8, theta* = 0.09036, z = 502.89159
xb: [94.2798, 200.5877, 11.9271, 145.8374, 84.8638, 50.2596, 23.1571, 1.497, 0.004, 0.4931]

Iteracion 7: q = 19, B[p] = 2, theta* = 41.0, z = 502.83735
xb: [94

In [28]:
# Vector c
c3 = np.array([-86, -76, -64, -49, -36, -49, -1, -79, -82, -99, 0, -95, -18, -95, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Matriz A
A3 = np.array([
    [28, 49, 53, 65, 75, 50, 64, 50, 98, 10, 79, 64, 46, 80, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [27, 36, 66, 90, 88, 88, 15, 35, 52, 26, 81, 5, 54, 96, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [26, 85, 0, 20, 53, 45, 20, 99, 74, 97, 24, 66, 52, 34, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0],
    [2, 38, 1, 93, 58, 17, 23, 93, 30, 65, 6, 73, 96, 65, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0],
    [83, 96, 5, 3, 98, 15, 34, 70, 73, 5, 1, 86, 8, 49, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
    [86, 31, 62, 64, 5, 14, 82, 5, 48, 60, 62, 60, 64, 88, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
    [31, 80, 32, 69, 27, 57, 68, 96, 33, 64, 48, 45, 65, 99, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
    [91, 36, 20, 98, 97, 3, 8, 67, 50, 83, 98, 46, 93, 42, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
    [95, 74, 60, 53, 5, 95, 12, 65, 55, 82, 50, 80, 82, 75, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
    [85, 75, 23, 71, 4, 84, 27, 98, 48, 72, 31, 65, 53, 98, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]
])

# Vector b
b3 = np.array([810, 758, 694, 659, 625, 730, 813, 831, 882, 833])

opt, zeta, acotado, factible = SimplexFaseI(A3,b3,c3)
if acotado and factible:
    print("vb:",opt)
    print("z*:",zeta)

Inicio del Simplex Primal con la regla de Bland
Fase I
Iteracion 1: q = 13, B[p] = 25, theta* = 7.89583, z = 7635.0
xb: [810.0, 758.0, 694.0, 659.0, 625.0, 730.0, 813.0, 831.0, 882.0, 833.0]

Iteracion 2: q = 11, B[p] = 29, theta* = 0.63459, z = 1902.625
xb: [178.3333, 7.8958, 425.5417, 145.7708, 238.1042, 35.1667, 31.3125, 499.375, 289.8125, 59.2083]

Iteracion 3: q = 7, B[p] = 30, theta* = 0.07594, z = 1552.21429
xb: [140.3639, 7.8628, 384.7827, 101.5944, 185.1492, 0.6346, 6.0282, 471.5722, 241.5244, 21.1992]

Iteracion 4: q = 4, B[p] = 27, theta* = 0.88746, z = 1500.33222
xb: [136.5611, 7.8332, 375.822, 93.7477, 178.0928, 0.6717, 0.0759, 466.0208, 235.8405, 14.2473]

Iteracion 5: q = 0, B[p] = 33, theta* = 0.20219, z = 897.82705
xb: [57.346, 6.9158, 266.2857, 0.8875, 20.1554, 1.9345, 0.1804, 353.3756, 192.3883, 8.276]

Iteracion 6: q = 6, B[p] = 28, theta* = 0.00801, z = 827.36586
xb: [57.5552, 6.7508, 255.5767, 0.964, 1.504, 1.8709, 0.2936, 329.8232, 182.9068, 0.2022]

Iteracion 7: